<a href="https://www.nvidia.com/dli"> <img src="images/DLI Header.png" alt="Header" style="width: 400px;"/> </a>

# Multidimensional Grids and Shared Memory for CUDA Python with Numba

In this section we will introduce a few more intermediate techniques for programming in CUDA with Numba. First we will introduce how CUDA provides the ability to create 2 and 3 dimensional blocks and grids which can ease programming when working with 2 and 3 dimensional data. Next, we will introduce an on-chip, programmer managed memory space called **shared memory** which can be used for very fast communication between threads within a given block. You will have a chance to use both of these techniques to optimize a 2D matrix multiplication kernel.

This section also provides an appendix with an example of reducing **shared memory bank conflicts** for a matrix transpose algorithm, with a link to resources for further study.

## Objectives

By the time you complete the required parts of this section you will be able to:

* Do GPU accelerated parallel work on multidimensional data sets using multi dimensional blocks and grids.
* Use shared memory to cache data on chip and reduce slow global memory accesses.

## 2 and 3 Dimensional Blocks and Grids

Both grids and blocks can be configured to contain a 2 or 3 dimensional collection of blocks or threads, respectively. This is done mostly as a matter of convenience for programmers who often work with 2 or 3 dimensional datasets. Here is a very trivial example to highlight the syntax. You may need to read *both* the kernel definition and its launch before the concept makes sense.

In [1]:
import numpy as np
from numba import cuda

In [2]:
A = np.zeros(16).reshape(4, 4).astype(np.int32)
d_A = cuda.to_device(A)

@cuda.jit
def add_2D_coordinates(A):
    # By passing `2`, we get the thread's unique x and y coordinates in the 2D grid
    x, y = cuda.grid(2)
    
    A[y][x] = x + y

In [3]:
# Here we create a 2D grid with 4 blocks in a 2x2 structure, each with 4 threads in a 2x2 structure
# by using a Python tuple to signify grid and block dimensions.
blocks = (2, 2)
threads_per_block = (2, 2)

add_2D_coordinates[blocks, threads_per_block](d_A)
print(d_A.copy_to_host())

[[0 1 2 3]
 [1 2 3 4]
 [2 3 4 5]
 [3 4 5 6]]


### Aside About Exercises

It's important always to remember that GPUs perform well when they are given large enough grids to support SMs in always having additional work to do while they wait on operations with latency to expire. With that in mind it's worth mentioning that several of the exercises in this section will ask you to write kernels that do not follow this best practice. This is done to give you a chance to focus on the syntax and some of the basic patterns of working in multiple dimensions, which tends to take a bit of orientation, even without doing meaningful work.

### Exercise: Add 2D Matrices on the GPU

In this exercise you will modify a kernel definition and its launch configuration to perform 2D matrix addition in parallel. To start with you'll be writing a naive kernel that will only work when launched with a grid whose shape matches that of the data. If you get stuck feel free to refer to [the solution](../edit/solutions/add_matrix_solution.py).

In [4]:
# This function is written to add two 1D vectors. Refactor it to add 2D matrices
@cuda.jit
def add_matrix(A, B, C):
    i, j = cuda.grid(2)
    
    C[i,j] = A[i,j] + B[i,j]

In [5]:
# Do not modify the values in this cell, which defines 2D matrices of size 36*36
A = np.arange(36*36).reshape(36, 36).astype(np.int32)
B = A * 2
C = np.zeros_like(A)
d_A = cuda.to_device(A)
d_B = cuda.to_device(B)
d_C = cuda.to_device(C)

In [6]:
# Refactor the launch configuration to use a 2D grid with 2D blocks
blocks = (6,6)
threads_per_block = (6,6)

# This launch will throw a Typing error until refactor the definition above to operate on 2D arrays
add_matrix[blocks, threads_per_block](d_A, d_B, d_C)

In [7]:
from numpy import testing
output = d_C.copy_to_host()
solution = A+B
# This assertion will fail unles the output and solution are equal
testing.assert_array_equal(output, solution)

### Exercise: Multiply 2D Matrices on the GPU

In this exercise you'll complete the logic of a kernel that calculates one element for a 2D [matrix multiplication](https://en.wikipedia.org/wiki/Matrix_multiplication) operation. Like the matrix add kernel you just wrote, this kernel is also naive in that it requires grid dimensions to match that of the passed in matrices. Refer to the [solution](../edit/solutions/matrix_multiply_solution.py) if you get stuck.

In [8]:
import numpy as np
from numba import cuda

@cuda.jit
def mm(a, b, c):
    row, column = cuda.grid(2)
    sum = 0
    
    for i in range(a.shape[0]):
        sum += a[row][i] * b[i][column]
        
    c[row][column] = sum

In [9]:
# Do not modify the values in this cell
a = np.arange(16).reshape(4,4).astype(np.int32)
b = np.arange(16).reshape(4,4).astype(np.int32)
c = np.zeros_like(a)

d_a = cuda.to_device(a)
d_b = cuda.to_device(b)
d_c = cuda.to_device(c)

grid = (2,2)
block = (2,2)
mm[grid, block](d_a, d_b, d_c)

In [10]:
from numpy import testing
solution = a@b
output = d_c.copy_to_host()
# This assertion will fail until you successfully implement the kernel
testing.assert_array_equal(output, solution)

## Striding in Multiple Dimensions

Just as we can use Numba's `cuda.gridsize(1)` to obtain the total number of threads in a grid, so too we can use `cuda.gridsize(2)` to get variables for the total number of threads in each each direction for a 2D grid. This can be useful, for example, when we have a 2D dataset larger than our grid where we would wish for each thread to stride over the grid in a loop in order to cover all necessary work.

Just like with our 1D grid stride loop, this technique also allows flexibility in the sizes of our grids and blocks, regardless of the shape of our data. The following example demonstrates the use of a 2D grid stride loop, ultimately printing information about which threads in the grid are working on which elements in the matrix.

In [11]:
import numpy as np
from numba import cuda
@cuda.jit
def add_2D_coordinates_stride(A):

    grid_y, grid_x = cuda.grid(2)
    # By passing `2`, we get the grid size in both the x an y dimensions
    stride_y, stride_x = cuda.gridsize(2)
    
    for data_i in range(grid_x, A.shape[0], stride_x):
        for data_j in range(grid_y, A.shape[1], stride_y):
            A[data_i][data_j] = grid_x + grid_y

Here we create a 2D grid with 6 blocks in a 3x2 structure, each with 6 threads in a 3x2 structure. The grid is both smaller than our total data set, and has a shape that does not fit evenly into the dataset's dimensions. Still, the kernel is able to access each element in the data. After running the cell, play around with both the data's shape, as the grid's. Try to predict what the output matrix's values will be before running the code.

In [12]:
A = np.zeros(55).reshape(11, 5).astype(np.int32)
d_A = cuda.to_device(A)

blocks = (3, 2)
threads_per_block = (3, 2)

# With this configuration, `stride_x` will be 9, and `stride_y` will be 4
add_2D_coordinates_stride[blocks, threads_per_block](d_A)
print(d_A.copy_to_host())

[[0 1 2 3 4]
 [1 2 3 4 5]
 [2 3 4 5 6]
 [3 4 5 6 7]
 [0 1 2 3 4]
 [1 2 3 4 5]
 [2 3 4 5 6]
 [3 4 5 6 7]
 [0 1 2 3 4]
 [1 2 3 4 5]
 [2 3 4 5 6]]


### Exercise: Add 2D Matrices Larger than the Grid Size

In this exercise you will modify the naive matrix addition kernel above to be able to work on datasets of an arbitrary size. If you get stuck feel free to refer to [the solution](../edit/solutions/add_matrix_stride_solution.py).

In [16]:
# Currently this kernel will only work correctly when passed matrices that are of the same size as the grid.
# Refactor using a strid in 2D so that it can work on data sets of an arbitrary size.
@cuda.jit
def add_matrix_stride(A, B, C):
    grid_x,grid_y = cuda.grid(2)
    stride_y, stride_x = cuda.gridsize(2)
    
    for data_i in range(grid_x, A.shape[0], stride_x):
        for data_j in range(grid_y, A.shape[1], stride_y):
            C[data_i,data_j] = A[data_i,data_j] + B[data_i,data_j]

In [17]:
# Please don't modify the values in this cell. They create a scenario where the data is
# larger than the grid size.
A = np.arange(64*64).reshape(64, 64).astype(np.int32)
B = A * 2
C = np.zeros_like(A)
d_A = cuda.to_device(A)
d_B = cuda.to_device(B)
d_C = cuda.to_device(C)

blocks = (6,6)
threads_per_block = (6,6)

add_matrix_stride[blocks, threads_per_block](d_A, d_B, d_C)

In [18]:
from numpy import testing
output = d_C.copy_to_host()
solution = A+B
# This assertion will fail unles the output and solution are equal
testing.assert_array_equal(output, solution)

### Exercise: Multiply 2D Matrices Larger than the Grid Size

In this exercise you will complete a matrix multiplication kernel that will be able to work with arbitrary grid and data set shapes. You only need to work on the two lines containing the `TODO`s, using the `grid_` and `stride_` values to correctly map the work of the executing kernel into the data. Refer to the [solution](../edit/solutions/matrix_multiply_stride_solution.py) if you get stuck.

In [25]:
import numpy as np
from numba import cuda
@cuda.jit
def mm_stride(A, B, C):

    grid_row, grid_column = cuda.grid(2)
    stride_row, stride_column = cuda.gridsize(2)
    
    for data_row in range(grid_row, A.shape[0], stride_row): # TODO: replace 0 with values that will correctly set data_row
        for data_column in range(grid_column, B.shape[1], stride_column): # TODO: replace 0 with values that will correctly set data_column
            sum = 0
            for i in range(A.shape[1]): # B.shape[0] would also be okay here
                sum += A[data_row][i] * B[i][data_column]
                
            C[data_row][data_column] = sum

In [26]:
# Please do not modify this cell. The strange dimensions of this data, and
# the grid below are being set to make sure your kernel correctly handles arbitrary
# data and grid sizes.

a = np.arange(12).reshape(3,4).astype(np.int32)
b = np.arange(24).reshape(4,6).astype(np.int32)
c = np.zeros((a.shape[0], b.shape[1])).astype(np.int32)

d_a = cuda.to_device(a)
d_b = cuda.to_device(b)
d_c = cuda.to_device(c)

ts = (4, 3)
bs = (3, 7)

mm_stride[bs, ts](d_a, d_b, d_c)

In [27]:
from numpy import testing
solution = a@b
output = d_c.copy_to_host()
# This assertion will fail until you correctly update the kernel above.
testing.assert_array_equal(output, solution)

## Shared Memory

So far we have been differentiating between host and device memory, as if device memory were a single kind of memory. But in fact, CUDA has an even more fine-grained [memory hierarchy](https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#memory-hierarchy). The device memory we have been utilizing thus far is called **global memory** which is available to any thread or block on the device, can persist for the lifetime of the application, and is a relatively large memory space.

As a final topic we will discuss how to utilize a region of on-chip device memory called **shared memory**. Shared memory is a programmer defined cache of limited size that [depends on the GPU](https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#compute-capabilities) being used and is **shared** between all threads in a block. It is a scarce resource, cannot be accessed by threads outside of the block where it was allocated, and does not persist after a kernel finishes executing. Shared memory however has a much higher bandwidth than global memory and can be used to great effect in many kernels, especially to optimize performance.

Here are a few common use cases for shared memory:

 * Caching memory read from global memory that will need to be read multiple times within a block.
 * Buffering output from threads so it can be coalesced before writing it back to global memory.
 * Staging data for scatter/gather operations within a block.

### Shared Memory Syntax

Numba provides [functions](https://numba.pydata.org/numba-doc/dev/cuda/memory.html#shared-memory-and-thread-synchronization) for allocating shared memory as well as for synchronizing between threads in a block, which is often necessary after parallel threads read from or write to shared memory.

When declaring shared memory, you provide the shape of the shared array, as well as its type, using a [Numba type](https://numba.pydata.org/numba-doc/dev/reference/types.html#numba-types). **The shape of the array must be a constant value**, and therefore, you cannot use arguments passed into the function, or, provided variables like `numba.cuda.blockDim.x`, or the calculated values of `cuda.griddim`. Here is a convoluted example to demonstrate the syntax with comments pointing out the movement from host memory to global device memory, to shared memory, back to global device memory, and finally back to host memory:

In [28]:
import numpy as np
from numba import types, cuda

@cuda.jit
def swap_with_shared(x, y):
    # Allocate a 4 element vector containing int32 values in shared memory.
    temp = cuda.shared.array(4, dtype=types.int32)
    
    idx = cuda.grid(1)
    
    # Move an element from global memory into shared memory
    temp[idx] = x[idx]
    
    # cuda.syncthreads will force all threads in the block to synchronize here, which is necessary because...
    cuda.syncthreads()
    #...the following operation is reading an element written to shared memory by another thread.
    
    # Move an element from shared memory back into global memory
    y[idx] = temp[cuda.blockDim.x - cuda.threadIdx.x - 1] # swap elements

In [29]:
x = np.arange(4).astype(np.int32)
y = np.zeros_like(x)

# Move host memory to device (global) memory
d_x = cuda.to_device(x)
d_y = cuda.to_device(y)

swap_with_shared[1, 4](d_x, d_y)

In [30]:
# Move device (global) memory back to the host
d_y.copy_to_host()

array([3, 2, 1, 0], dtype=int32)

### A Shared Memory Example: Matrix Transposition

As an example of the power of shared memory, and as a further example in 2D CUDA programming, let's write a matrix transpose kernel that takes a 2D array in row-major order and puts it in column-major order. (This is based on Mark Harris' [Efficient Matrix Transpose](https://devblogs.nvidia.com/parallelforall/efficient-matrix-transpose-cuda-cc/) blog post, but uses a slightly simpler algorithm).

In this example, we will be using shared memory as a mechanism to allow both coalesced reads and coalesced writes to and from global memory. This would typically not be possible for a transposition algorithm, but because shared memory access is so fast, we can make "non-coalesced" reads from or writes to it with no performance penalty in order to then read and/or write to or from global memory in a coalesced manner. The example will demonstrate this.

#### Matrix Transposition Without Shared Memory

Before showing the shared memory implementation, let's first do a naive approach where we let each thread read and write individual elements independently using only global memory. In this way, when you see the shared memory example, you can focus on how shared memory impacts the algorithm, as opposed to the basics of the algorithm itself:

In [31]:
@cuda.jit
def transpose(a_in, a_out):
    # Explicitly calculate indices rather than using cuda.grid(2)
    row = cuda.blockIdx.x * cuda.blockDim.x + cuda.threadIdx.x
    col = cuda.blockIdx.y * cuda.blockDim.y + cuda.threadIdx.y

    a_out[row, col] = a_in[col, row]

In [32]:
size = 16384
a_in = cuda.to_device(np.arange(size*size, dtype=np.int32).reshape((size, size)))
a_out = cuda.device_array_like(a_in)

print(a_in.copy_to_host())

[[        0         1         2 ...     16381     16382     16383]
 [    16384     16385     16386 ...     32765     32766     32767]
 [    32768     32769     32770 ...     49149     49150     49151]
 ...
 [268386304 268386305 268386306 ... 268402685 268402686 268402687]
 [268402688 268402689 268402690 ... 268419069 268419070 268419071]
 [268419072 268419073 268419074 ... 268435453 268435454 268435455]]


In [33]:
threads_per_block = (32, 32)
blocks_per_grid = (int(size/threads_per_block[0]), int(size/threads_per_block[1]))

In [34]:
%timeit transpose[blocks_per_grid, threads_per_block](a_in, a_out); cuda.synchronize()

25.2 ms ± 520 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [35]:
print(a_out.copy_to_host())

[[        0     16384     32768 ... 268386304 268402688 268419072]
 [        1     16385     32769 ... 268386305 268402689 268419073]
 [        2     16386     32770 ... 268386306 268402690 268419074]
 ...
 [    16381     32765     49149 ... 268402685 268419069 268435453]
 [    16382     32766     49150 ... 268402686 268419070 268435454]
 [    16383     32767     49151 ... 268402687 268419071 268435455]]


#### Matrix Transposition With Shared Memory

Now let's execute the algorithm using shared memory. To do so:

1. Each block will create a 32x32 element shared memory array
2. Each block will make a coalesced read from the input array in global memory into the shared memory array
3. Before writing the values back to global memory, each thread in the block will wait for all other threads in the block to complete their reads using a thread sync
4. We make a coalesced write to global memory from shared memory, writing in a transposed order. We will do this in two steps, first transposing the location of the shared memory tile within the total input array (4a below), and then, transposing the locations of the elements within the tile before writing them back to global memory (4b below).

This method allows us to make the coalesced reads and writes to and from global memory since we can perform the transposition within the shared memory space where there is no performance penalty for non-contiguous reads and writes within shared memory arrays.

In [36]:
import numba.types

@cuda.jit
def tile_transpose(a_in, a_out):
    # `tile_transpose` assumes it is launched with a 32x32 block dimension,
    # and that `a_in` is a multiple of these dimensions.
    
    # 1) Create 32x32 shared memory array.
    tile = cuda.shared.array((32, 32), numba.types.int32)

    # Compute offsets into global input array.
    row = cuda.blockIdx.x * cuda.blockDim.x + cuda.threadIdx.x
    col = cuda.blockIdx.y * cuda.blockDim.y + cuda.threadIdx.y
    
    # 2) Make coalesced read from global memory into shared memory array.
    # Note the use of local thread indices for the shared memory write,
    # and global offsets for global memory read.
    tile[cuda.threadIdx.y, cuda.threadIdx.x] = a_in[col, row]

    # 3) Wait for all threads in the block to finish updating shared memory.
    cuda.syncthreads()
    
    # 4a) Calculate transposed location for the shared memory array tile
    # to be written back to global memory...
    row = cuda.blockIdx.y * cuda.blockDim.y + cuda.threadIdx.x
    col = cuda.blockIdx.x * cuda.blockDim.x + cuda.threadIdx.y

    # 4b) ...Write back to global memory,
    # transposing each element within the shared memory array.
    a_out[col, row] = tile[cuda.threadIdx.x, cuda.threadIdx.y]

In [37]:
a_out = cuda.device_array_like(a_in)

%timeit tile_transpose[blocks_per_grid, threads_per_block](a_in, a_out); cuda.synchronize()
print(a_out.copy_to_host())

15.3 ms ± 603 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
[[        0     16384     32768 ... 268386304 268402688 268419072]
 [        1     16385     32769 ... 268386305 268402689 268419073]
 [        2     16386     32770 ... 268386306 268402690 268419074]
 ...
 [    16381     32765     49149 ... 268402685 268419069 268435453]
 [    16382     32766     49150 ... 268402686 268419070 268435454]
 [    16383     32767     49151 ... 268402687 268419071 268435455]]


That's a decent speed up on top of already accelerated code. (The [Efficient Matrix Transpose](https://devblogs.nvidia.com/parallelforall/efficient-matrix-transpose-cuda-cc/) blog post demonstrates a method to achieve an even higher speedup by having a thread in the block be responsible for multiple entries in a tile, which reduces the cost of indexing.)

## Assessment

The following exercise will require you to utilize everything you've learned so far. Unlike previous exercises, there will not be any solution code available to you, and, there are a couple additional steps you will need to take to "run the assessment" and get a score for your attempt(s). **Please read the directions carefully before beginning your work to ensure the best chance at successfully completing the assessment.**

### How to Run the Assessment

Take the following steps to complete this assessment:

1. Using the instructions that follow, work on the cells below as you usually would for an exercise.
2. When you are satisfied with your work, follow the instructions below to copy and paste code in into linked source code files. Be sure to save the files after you paste your work.
3. Return to the browser tab you used to launch this notebook, and click on the **"Assess"** button. After a few seconds a score will be generated along with a helpful message.

You are welcome to click on the **Assess** button as many times as you like, so feel free if you don't pass the first time to make additional modifications to your code and repeat steps 1 through 3. Good luck!

![Run the assessment](images/run_assess_task.png)

### Matrix Multiply with Shared Memory

In this exercise you will complete a matrix mulitply kernel that will use shared memory to cache values from the input matrices so that they only need be accessed from global memory once, after which calculations for a thread's output element can utilize the cached values.This purpose of this assessment is to test your ability to reason about a 2D parallel problem and utilize shared memory. This particular problem doesn't have a ton of arithmetic intensity,  and we are not going to use a huge dataset so we will likely not see big speedups vs. the very simple CPU version. However, the ability to use the techniques asked of you will provide you ability in a wide number of situations where you will genuinely wish to accelerate some program involving a 2D dataset.

To keep the focus on shared memory, this problem assumes input vectors of MxN and NxM dimensions with NxN threads per block and M/N blocks per grid. This means that shared memory caches with elements equal to the number of threads per block will be sufficient to provide all elements from the input matrices necessary for the calculations, and that no grid striding will be required.

The following images shows the input matrices, the output matrix, a region of the output matrix that a block will calculate values for, the regions in the input matrices that this block will cache, and also, the output element and input elements for a single thread in that block:

![matrix multiply diagram](images/mm_image.png)

The shared memory caches have already been allocated in the kernel, your task is twofold:
1. Use each thread in the block to populate one element in each of the caches.
2. Use the shared memory caches in calculating each thread's `sum` value.

Be sure to do any thread synchronizing that might be required to avoid cached values written by other threads not yet being available.

Additionally, when running the assessment, your code will be assessed against several different input sizes, so you will need to write a kernel that can robustly handle arbitrary input sizes. After successfully solving for the input values below, and before running the assessment, it is recommended you modify the input values (see comment in the cell below) to make sure your code works when changing input sizes.

In [38]:
# You do not need to edit the values in this cell, however, before running the assessment
# it is recommended you try your code against different input sizes to make sure it can
# handle arbitrary input sizes.
M = 128
N = 32

# Consider using this value for N before running the assessment
# to make sure your code handles arbitrary input sizes.
# N = 8

# Input vectors of MxN and NxM dimensions
a = np.arange(M*N).reshape(M,N).astype(np.int32)
b = np.arange(M*N).reshape(N,M).astype(np.int32)
c = np.zeros((M, M)).astype(np.int32)

d_a = cuda.to_device(a)
d_b = cuda.to_device(b)
d_c = cuda.to_device(c)

# NxN threads per block, in 2 dimensions
block_size = (N,N)
# MxM/NxN blocks per grid, in 2 dimensions
grid_size = (int(M/N),int(M/N))

After making any modifications to `mm_shared` in the cell below, and before running the assessment, paste this cell's content into [**`assessment/definition.py`**](../edit/assessment/definition.py)

In [48]:
import numpy as np
from numba import cuda, types
@cuda.jit
def mm_shared(a, b, c):
    column, row = cuda.grid(2)
    sum = 0

    # `a_cache` and `b_cache` are already correctly defined
    a_cache = cuda.shared.array(block_size, types.int32)
    b_cache = cuda.shared.array(block_size, types.int32)

    # TODO: use each thread to populate one element each a_cache and b_cache
    a_cache[cuda.threadIdx.x, cuda.threadIdx.y] = a[row][col]
    b_cache[cuda.threadIdx.y, cuda.threadIdx.x] = b[col][row]

    for i in range(a.shape[1]):
        a_cache[cuda.threadIdx.x, cuda.threadIdx.y] = a[row, cuda.threadIdx.y + i * N]
        b_cache[cuda.threadIdx.x, cuda.threadIdx.y] = b[cuda.threadIdx.x + i * N, col]

        cuda.syncthreads()

        for j in range(N):
            sum += a_cache[cuda.threadIdx.x, j] * b_cache[j, cuda.threadIdx.y]

        # Wait until all threads finish computing
        cuda.syncthreads()
        
    c[row][column] = sum

In [49]:
# There's no need to update this kernel launch
mm_shared[grid_size, block_size](d_a, d_b, d_c)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
Untyped global name 'col': cannot determine Numba type of <class 'numba.ir.UndefinedType'>

File "<ipython-input-48-4e6f37abebb2>", line 13:
def mm_shared(a, b, c):
    <source elided>
    # TODO: use each thread to populate one element each a_cache and b_cache
    a_cache[cuda.threadIdx.x, cuda.threadIdx.y] = a[row][col]
    ^


In [50]:
# Do not modify the contents in this cell
from numpy import testing
solution = a@b
output = d_c.copy_to_host()
# This assertion will fail until you correctly update the kernel above.
testing.assert_array_equal(output, solution)

AssertionError: 
Arrays are not equal

(mismatch 100.0%)
 x: array([[ 15134720,  23261184,  31387648, ..., 770371584, 777973760,
        267055104],
       [ 15142016,  23272576,  31403136, ..., 770751616, 778349696,...
 y: array([[  1333248,   1333744,   1334240, ...,   1395248,   1395744,
          1396240],
       [  3364864,   3366384,   3367904, ...,   3554864,   3556384,...

## Summary

Now that you have completed this section you are able to:

* Do GPU accelerated parallel work on multidimensional data sets using multi dimensional blocks and grids.
* * Use shared memory to cache data on chip and reduce slow global memory accesses.

## Download Content

To download the contents of this notebook, execute the following cell and then click the download link below. Note: If you run this notebook on a local Jupyter server, you can expect some of the file path links in the notebook to be broken as they are shaped to our own platform. You can still navigate to the files through the Jupyter file navigator.

In [ ]:
!tar -zcvf section3.tar.gz .

[Download the files for this section.](files/section3.tar.gz)

## Appendix: Bank Conflict Free Matrix Transpose

Shared memory is stored in **banks**. There are 32 banks available for storing shared memory, and memory reads and writes that do not use the same memory bank can perform simultaneously. When parallel threads attempt to access memory in the same bank, we call this a **bank conflict**, which results in the operations being serialized. Even with bank conflicts, shared memory is very fast, but creating memory access patterns that avoid bank conflict are a way to further optimize your applications.

We will only give an example here, but for more details, consider the [Efficient Matrix Transpose](https://devblogs.nvidia.com/parallelforall/efficient-matrix-transpose-cuda-cc/) blog post.

In [ ]:
from numba import cuda, types
import numpy as np

TILE_DIM = 32
BLOCK_ROWS = 8
TILE_DIM_PADDED = TILE_DIM + 1  # Read Mark Harris' blog post to find out why this improves performance!
                                # https://devblogs.nvidia.com/parallelforall/efficient-matrix-transpose-cuda-cc/

@cuda.jit
def tile_transpose_no_bank_conflict(a_in, a_out):
    # THIS CODE ASSUMES IT IS RUNNING WITH A BLOCK DIMENSION OF (TILE_SIZE x TILE_SIZE)
    # AND INPUT IS A MULTIPLE OF TILE_SIZE DIMENSIONSx
    tile = cuda.shared.array((TILE_DIM, TILE_DIM_PADDED), types.int32)

    x = cuda.blockIdx.x * TILE_DIM + cuda.threadIdx.x
    y = cuda.blockIdx.y * TILE_DIM + cuda.threadIdx.y
    
    for j in range(0, TILE_DIM, BLOCK_ROWS):
        tile[cuda.threadIdx.y + j, cuda.threadIdx.x] = a_in[y + j, x] # move tile into shared memory

    cuda.syncthreads()  # wait for all threads in the block to finish updating shared memory

    # Compute transposed offsets
    x = cuda.blockIdx.y * TILE_DIM + cuda.threadIdx.x
    y = cuda.blockIdx.x * TILE_DIM + cuda.threadIdx.y

    for j in range(0, TILE_DIM, BLOCK_ROWS):
        a_out[y + j, x] = tile[cuda.threadIdx.x, cuda.threadIdx.y + j];


In [ ]:
size = 8192
a_in = cuda.to_device(np.arange(size*size, dtype=np.int32).reshape((size, size)))
a_out = cuda.device_array_like(a_in)

print(a_in.copy_to_host())

In [ ]:
grid_shape = (int(size/TILE_DIM), int(size/TILE_DIM))

%timeit tile_transpose_no_bank_conflict[grid_shape,(TILE_DIM, BLOCK_ROWS)](a_in, a_out); cuda.synchronize()
print(a_out.copy_to_host())

<a href="https://www.nvidia.com/dli"> <img src="images/DLI Header.png" alt="Header" style="width: 400px;"/> </a>